In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/Deep_learning_final

/content/drive/MyDrive/Colab Notebooks/Deep_learning_final


Table by animal

| **Patch Size** | **Cat**   | **Deer**  | **Dog**   | **Frog**  |
|----------------|-----------|-----------|-----------|-----------|
| **Size 3**     | 10.25%    | 9.41%     | 11.20%    | 9.37%     |
| **Size 5**     | 10.12%    | 10.23%    | 10.66%    | 9.16%     |
| **Size 7**     | 9.88%     | 10.49%    | 10.79%    | 9.86%     |
| **Size 16**    | 9.52%     | 12.13%    | 13.67%    | 9.35%     |

Table by size

| **Target Class** | **Size 3** | **Size 5** | **Size 7** | **Size 16** |
|------------------|------------|------------|------------|-------------|
| **Cat**          | 10.25%     | 10.12%     | 9.88%      | 9.52%       |
| **Deer**         | 9.41%      | 10.23%     | 10.49%     | 12.13%      |
| **Dog**          | 11.20%     | 10.66%     | 10.79%     | 13.67%      |
| **Frog**         | 9.37%      | 9.16%      | 9.86%      | 9.35%       |

In [ ]:
# Function to create a random patch
def create_patch(patch_size=(3, 3)):
    patch = torch.randn(3, *patch_size, requires_grad=True, device=device)
    return patch

# Function to apply the patch to images
def place_patch(img, patch):
    batch_size, _, h, w = img.size()
    ph, pw = patch.size(1), patch.size(2)
    for i in range(batch_size):
        x_offset = torch.randint(0, h - ph + 1, (1,)).item()
        y_offset = torch.randint(0, w - pw + 1, (1,)).item()
        img[i, :, x_offset:x_offset+ph, y_offset:y_offset+pw] = patch
    return img

# Training function for adversarial patch
def patch_training_step(model, patch, target_class=None, dataloader=None, optimizer=None, criterion=None):
    model.train()
    total_loss = 0
    for images, _ in dataloader:
        images = images.to(device)
        optimizer.zero_grad()
        patched_images = place_patch(images, patch)
        outputs = model(patched_images)
        if target_class is not None:
            labels = torch.full((images.size(0),), target_class, dtype=torch.long, device=device)
        else:
            labels = torch.randint(0, 10, (images.size(0),), device=device)  # Random class for untargeted attack
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)

# Function to train the adversarial patch
def train_adversarial_patch(model, patch, target_class=None, num_epochs=10):
    patch_optimizer = optim.Adam([patch], lr=0.01)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        loss = patch_training_step(model, patch, target_class, trainloader, patch_optimizer, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")

# Function to evaluate the success rate of the adversarial patch
def evaluate_patch_success_rate(model, patch, dataloader, target_class=None):
    model.eval()
    successful_attacks = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            patched_images = place_patch(images.clone(), patch)
            outputs = model(patched_images)
            _, predicted = outputs.max(1)

            # Count successful targeted attacks (where prediction matches the target class)
            if target_class is not None:
                successful_attacks += (predicted == target_class).sum().item()
            total_samples += labels.size(0)

    success_rate = 100 * successful_attacks / total_samples
    print(f"Targeted Attack Success Rate: {success_rate:.2f}%")
    return success_rate

# Function to visualize the patch
def visualize_patch(patch):
    patch = patch.detach().cpu().numpy()
    patch = np.clip((patch - patch.min()) / (patch.max() - patch.min()), 0, 1)

    plt.figure(figsize=(2, 2))
    plt.imshow(np.transpose(patch, (1, 2, 0)))  # [C, H, W] -> [H, W, C]
    plt.title("Adversarial Patch")
    plt.axis('off')
    plt.show()

# Function to visualize misclassified patched images
def visualize_misclassified_patched_images(model, patch, dataloader, classes, target_class, num_examples=5):
    model.eval()
    examples_shown = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        patched_images = place_patch(images.clone(), patch)
        with torch.no_grad():
            original_preds = model(images).argmax(dim=1)
            patched_preds = model(patched_images).argmax(dim=1)
        for i in range(images.size(0)):
            if patched_preds[i] == target_class:  # Targeted attack
                original_image = (images[i].cpu().detach().numpy().transpose(1, 2, 0) * 0.5) + 0.5
                patched_image = (patched_images[i].cpu().detach().numpy().transpose(1, 2, 0) * 0.5) + 0.5

                fig, axes = plt.subplots(1, 2, figsize=(4, 4))
                axes[0].imshow(np.clip(original_image, 0, 1))
                axes[0].set_title(f"Original Image\nTrue Label: {classes[labels[i].item()]}\nPredicted: {classes[original_preds[i].item()]}")

                axes[1].imshow(np.clip(patched_image, 0, 1))
                axes[1].set_title(f"Image with Adversarial Patch\nTrue Label: {classes[labels[i].item()]}\nPredicted: {classes[patched_preds[i].item()]}")

                plt.show()
                examples_shown += 1
                if examples_shown >= num_examples:
                    return

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from torchvision import models
num_classes =10

# CIFAR-10 class names
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Targeted classes (plane, car, bird)
target_classes = [3,4,5,6]  # Corresponding to 'cat', 'deer', 'dog', 'frog'

# Different patch sizes
patch_sizes = [(3, 3), (5, 5), (7, 7), (16, 16)]

#load resnet
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("cifar_resnet18_pretrained.pth"))
model = model.to(device)

# Train and evaluate patches for each target class
for target_class in target_classes:
    for patch_size in patch_sizes:
        print(f"\nTraining for patch size: {patch_size} with target class: {classes[target_class]}")
        patch = create_patch(patch_size)
        train_adversarial_patch(model, patch, target_class=target_class, num_epochs=20)

        # Save the patch
        torch.save(patch, f'adversarial_patch_{patch_size[0]}x{patch_size[1]}_target_{target_class}.pth')

        # Visualize the patch
        print(f"Visualizing patch of size {patch_size} for target class {classes[target_class]}")
        visualize_patch(patch)

        # Evaluate attack success rate
        evaluate_patch_success_rate(model, patch, testloader, target_class=target_class)

        # Visualize misclassified examples with the adversarial patch
        visualize_misclassified_patched_images(model, patch, testloader, classes, target_class, num_examples=5)


Output hidden; open in https://colab.research.google.com to view.